In [ ]:
pip install pulp matplotlib

In [3]:
import json
import numpy as np
from tabulate import tabulate
from google.colab import drive
import pulp
import matplotlib.pyplot as plt
import numpy as np
drive.mount('/content/drive')

Mounted at /content/drive


### Матрицы для тестирования:

In [ ]:
{
 "matrix": [
     [0, 1, -1],
     [0, -1, 0],
     [1, 0, -1]
 ]
}

In [ ]:
{
 "matrix": [
     [10, 4, 11, 7],
     [7, 6, 8, 20],
     [6, 2, 1, 11]
 ]
}

In [ ]:
{
 "matrix": [
     [4, 5, 6, 7],
     [3, 4, 6, 5],
     [7, 6, 10, 8],
     [8, 5, 4, 3]
 ]
}

In [ ]:
{
 "matrix": [
     [8, 9, 9, 4],
     [6, 5, 8, 7],
     [3, 4, 8, 6],
     [8, 9, 9, 4]
 ]
}

In [ ]:
{
 "matrix": [
     [7, 6, 5, 4, 2],
     [5, 4, 3, 2, 3],
     [5, 6, 6, 3, 5],
     [2, 3, 3, 2, 4]
 ]
}

### Реализация упрощения платёжной матрицы путём анализа доминирующих стратегий

In [4]:
def print_matrix(matrix, title):
    print(f"\n{title}:\n")
    print(tabulate(matrix, tablefmt='fancy_grid'))

In [5]:
def add_absolute_min(matrix):
    min_negative = None
    for row in matrix:
        for val in row:
            if val < 0 and (min_negative is None or val < min_negative):
                min_negative = val
    if min_negative is not None:
        abs_min = abs(min_negative)
        gamma = abs_min
        modified_matrix = [[element + abs_min for element in row] for row in matrix]
        return modified_matrix, gamma
    else:
        return matrix, 0

In [6]:
def dominance_reduction(matrix, recursive_iteration=1):
    matrix = np.array(matrix)

    dominant_rows = []
    dominant_cols = []

    for i in range(matrix.shape[1]):
        for j in range(i + 1, matrix.shape[1]):
            if all(matrix[:, i] <= matrix[:, j]):
                dominant_cols.append(j)
            elif all(matrix[:, j] <= matrix[:, i]):
                dominant_cols.append(i)

    for i in range(matrix.shape[0]):
        for j in range(i + 1, matrix.shape[0]):
            if all(matrix[i, :] >= matrix[j, :]):
                dominant_rows.append(j)
            elif all(matrix[j, :] >= matrix[i, :]):
                dominant_rows.append(i)

    result_matrix = np.delete(np.delete(matrix, dominant_rows, axis=0), dominant_cols, axis=1)

    print_matrix(result_matrix, f"Итерация {recursive_iteration}")

    if not np.array_equal(matrix, result_matrix):
        result_matrix = dominance_reduction(result_matrix, recursive_iteration + 1)

    return result_matrix

In [7]:
file_path = '/content/drive/MyDrive/Primat/2023/game_matrix.json'

with open(file_path, 'r') as file:
    data = json.load(file)

matrix = data['matrix']
print_matrix(matrix, "Исходная матрица")
modified_matrix, gamma = add_absolute_min(matrix)
print_matrix(modified_matrix, f"Матрица с добавлением модуля минимального отрицательного числа (gamma={gamma})")
simplified_matrix = dominance_reduction(modified_matrix)
print_matrix(simplified_matrix, "Упрощенная матрица")


Исходная матрица:

╒═══╤════╤════╕
│ 0 │  1 │ -1 │
├───┼────┼────┤
│ 0 │ -1 │  0 │
├───┼────┼────┤
│ 1 │  0 │ -1 │
╘═══╧════╧════╛

Матрица с добавлением модуля минимального отрицательного числа (gamma=1):

╒═══╤═══╤═══╕
│ 1 │ 2 │ 0 │
├───┼───┼───┤
│ 1 │ 0 │ 1 │
├───┼───┼───┤
│ 2 │ 1 │ 0 │
╘═══╧═══╧═══╛

Итерация 1:

╒═══╤═══╕
│ 2 │ 0 │
├───┼───┤
│ 0 │ 1 │
├───┼───┤
│ 1 │ 0 │
╘═══╧═══╛

Итерация 2:

╒═══╤═══╕
│ 2 │ 0 │
├───┼───┤
│ 0 │ 1 │
╘═══╧═══╛

Итерация 3:

╒═══╤═══╕
│ 2 │ 0 │
├───┼───┤
│ 0 │ 1 │
╘═══╧═══╛

Упрощенная матрица:

╒═══╤═══╕
│ 2 │ 0 │
├───┼───┤
│ 0 │ 1 │
╘═══╧═══╛


### Поиск решение в чистых стратегиях. Определение оптимальной стратегии и соответствующей цены игры.

In [8]:
modified_matrix, gamma = add_absolute_min(matrix)
print_matrix(modified_matrix, f"Матрица с добавлением модуля минимального отрицательного числа (gamma={gamma})")

min_values_rows = tuple(min(row) for row in modified_matrix)
maxmin = max(min_values_rows)

max_values_columns = tuple(max(column) for column in zip(*modified_matrix))
minmax = min(max_values_columns)

print("Минимальные значения строк:", min_values_rows)
print("Максимальные значения столбцов:", max_values_columns)
print("Нижняя цена (максимин):", maxmin - gamma)
print("Верхняя цена (минимакс):", minmax - gamma)

if maxmin == minmax:
    print("\nРешение в чистых стратегиях существует.")
    adjusted_min_values_rows = tuple(0 if value != maxmin else 1 for value in min_values_rows)
    print("P(a):", adjusted_min_values_rows)

    adjusted_max_values_columns = tuple(0 if value != minmax else 1 for value in max_values_columns)
    print("P(b):", adjusted_max_values_columns)

    index_of_1_rows = adjusted_min_values_rows.index(1)
    index_of_1_columns = adjusted_max_values_columns.index(1)

    print("\nСедловая точка: (A[{}], B[{}])".format(index_of_1_columns + 1, index_of_1_rows + 1))
else:
    print("\nРешение в чистых стратегиях невозможно.")


Матрица с добавлением модуля минимального отрицательного числа (gamma=1):

╒═══╤═══╤═══╕
│ 1 │ 2 │ 0 │
├───┼───┼───┤
│ 1 │ 0 │ 1 │
├───┼───┼───┤
│ 2 │ 1 │ 0 │
╘═══╧═══╧═══╛
Минимальные значения строк: (0, 0, 0)
Максимальные значения столбцов: (2, 2, 1)
Нижняя цена (максимин): -1
Верхняя цена (минимакс): 0

Решение в чистых стратегиях невозможно.


### Применение симплекс-метода для поиска седловой точки в смешанных стратегиях. Определение смешанной стратегии и соответсвующей цены игры.

In [ ]:
matrix = data['matrix']
print_matrix(matrix, "Исходная матрица")
modified_matrix, gamma = add_absolute_min(matrix)
print_matrix(modified_matrix, f"Матрица с добавлением модуля минимального отрицательного числа (gamma={gamma})")

player_A = list(map(list, zip(*modified_matrix)))
for row in player_A:
    row.append(1)
print_matrix(player_A, "Математическая модель для игрока А -> min")

player_B = modified_matrix
for row in player_B:
    row.append(1)
print_matrix(player_B, "Математическая модель для игрока B -> max")


Исходная матрица:

╒═══╤════╤════╕
│ 0 │  1 │ -1 │
├───┼────┼────┤
│ 0 │ -1 │  0 │
├───┼────┼────┤
│ 1 │  0 │ -1 │
╘═══╧════╧════╛

Матрица с добавлением модуля минимального отрицательного числа (gamma=1):

╒═══╤═══╤═══╕
│ 1 │ 2 │ 0 │
├───┼───┼───┤
│ 1 │ 0 │ 1 │
├───┼───┼───┤
│ 2 │ 1 │ 0 │
╘═══╧═══╧═══╛

Математическая модель для игрока А -> min:

╒═══╤═══╤═══╤═══╕
│ 1 │ 1 │ 2 │ 1 │
├───┼───┼───┼───┤
│ 2 │ 0 │ 1 │ 1 │
├───┼───┼───┼───┤
│ 0 │ 1 │ 0 │ 1 │
╘═══╧═══╧═══╧═══╛

Математическая модель для игрока B -> max:

╒═══╤═══╤═══╤═══╕
│ 1 │ 2 │ 0 │ 1 │
├───┼───┼───┼───┤
│ 1 │ 0 │ 1 │ 1 │
├───┼───┼───┼───┤
│ 2 │ 1 │ 0 │ 1 │
╘═══╧═══╧═══╧═══╛


In [ ]:
player_A = np.array(player_A)
n = player_A.shape

coefficients = [1] * n[0]
goal = pulp.LpMinimize
lp_problem = pulp.LpProblem("Simplex_Method", goal)
x = [pulp.LpVariable(f"x{i}", lowBound=0) for i in range(1, len(coefficients) + 1)]
lp_problem += pulp.lpDot(coefficients, x)

num_rows_A, num_cols_A = player_A.shape
b_values = player_A[:, -1]

constraints_A = [{"coefs": player_A[i, :-1].tolist(), "type": "gte", "b": b_values[i]} for i in range(num_rows_A)]

for constraint in constraints_A:
    if constraint["type"] == "lte":
        lp_problem += pulp.lpDot(constraint["coefs"], x) <= constraint["b"]
    elif constraint["type"] == "gte":
        lp_problem += pulp.lpDot(constraint["coefs"], x) >= constraint["b"]
    elif constraint["type"] == "eq":
        lp_problem += pulp.lpDot(constraint["coefs"], x) == constraint["b"]

lp_problem.solve()

optimal_value = pulp.value(lp_problem.objective)
nu = 1 / optimal_value
optimal_values = [nu * pulp.value(var) if pulp.value(var) != 0 else 0 for var in x]

print("Оптимальные значения переменных игрока А:", optimal_values)
print("Цена игры:", nu - gamma)


Оптимальные значения переменных игрока А: [0.3333333333333333, 0.6666666666666666, 0]
Цена игры: -0.33333333333333337


In [ ]:
player_B = np.array(player_B)

m = player_B.shape
coefficients = [1] * (m[1] - 1)
goal = pulp.LpMaximize
lp_problem = pulp.LpProblem("Simplex_Method", goal)
x = [pulp.LpVariable(f"x{i}", lowBound=0) for i in range(1, len(coefficients) + 1)]
lp_problem += pulp.lpDot(coefficients, x)

num_rows_B, num_cols_B = player_B.shape
b_values = player_B[:, -1]

constraints_B = [{"coefs": player_B[i, :-1].tolist(), "type": "lte", "b": b_values[i]} for i in range(num_rows_B)]

for constraint in constraints_B:
    if constraint["type"] == "lte":
        lp_problem += pulp.lpDot(constraint["coefs"], x) <= constraint["b"]
    elif constraint["type"] == "gte":
        lp_problem += pulp.lpDot(constraint["coefs"], x) >= constraint["b"]
    elif constraint["type"] == "eq":
        lp_problem += pulp.lpDot(constraint["coefs"], x) == constraint["b"]

lp_problem.solve()

optimal_value = pulp.value(lp_problem.objective)
nu = 1 / optimal_value
optimal_values = [nu * pulp.value(var) if pulp.value(var) != 0 else 0 for var in x]

print("Оптимальные значения переменных игрока B:", optimal_values)
print("Цена игры:", nu - gamma)


Оптимальные значения переменных игрока B: [0, 0.3333333333333333, 0.6666666666666666]
Цена игры: -0.33333333333333337
